In [ ]:
#    “Copyright (C) 2024 Mississippi State University.
 
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
 
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
 
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.
 
# To inquire about relicensing, accessing more training data, collaborating with the author, or any general inquiry about the software, please contact Mississippi State University’s Office of Technology Management at otm@msstate.edu, (662) 325-9263.”


import os
import numpy as np
from skimage import filters
import matplotlib.pyplot as plt
import rasterio

# Define the number of locations
num_locations = 240
# Output parent directory for stacked images
output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM masked stacked map/'

# Create the output parent directory if it doesn't exist
os.makedirs(output_parent_directory, exist_ok=True)

# Initialize lists to store Otsu thresholds and masked stacked images
LUM_otsu_thresholds = []
LUM_masked_maps = []

# Initialize variables to accumulate histograms
histogram_bins = 256
histogram_range = (0, 1)
accumulated_non_zero_histogram = np.zeros(histogram_bins)
accumulated_all_histogram = np.zeros(histogram_bins)

# Initialize lists to store individual histograms
individual_non_zero_histograms = []

# Loop through each location index
for location_index in range(1, num_locations + 1):
    # Select the stacked image for the current location
    location_stacked_image = LUM_maps[location_index - 1]  # Adjust indexing as needed
    
    # Assuming each band contains reflectance values
    bands = location_stacked_image
    
    # Define band names
    band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']
    
    # Define a threshold for soil pixels using Otsu's method on NIR band
    nir_band = location_stacked_image[4]  # Assuming NIR band is index 4
    
    # Accumulate histogram for the NIR band including zero values
    hist_all, bin_edges_all = np.histogram(nir_band.flatten(), bins=histogram_bins, range=histogram_range)
    accumulated_all_histogram += hist_all
    
    # Remove zero values before applying Otsu's method
    non_zero_nir_band = nir_band[nir_band > 0]
    
    # Accumulate histogram for the non-zero NIR band
    hist_non_zero, bin_edges_non_zero = np.histogram(non_zero_nir_band, bins=histogram_bins, range=histogram_range)
    accumulated_non_zero_histogram += hist_non_zero
    
    # Store individual non-zero histograms
    individual_non_zero_histograms.append(hist_non_zero)
    
    # Apply Otsu's thresholding
    if len(non_zero_nir_band) > 0:  # Ensure there are non-zero values
        LUM_threshold_otsu = filters.threshold_otsu(non_zero_nir_band)
    else:
        LUM_threshold_otsu = 0
        print(f"Location {location_index}: No non-zero values in NIR band.")
    
    LUM_otsu_thresholds.append(LUM_threshold_otsu)
    
    # Print the Otsu threshold for the current location
    print(f"Location {location_index}: LUM Otsu Threshold for NIR Band = {LUM_threshold_otsu:.2f}")
    
    # Create binary mask for soil pixels based on Otsu's threshold
    soil_binary_mask = nir_band < LUM_threshold_otsu
    
    # Create a list to hold masked bands
    LUM_masked_bands = []
    
    # Loop through each band and mask soil pixels
    for i, band in enumerate(bands):
        # Mask soil pixels by setting them to NaN
        LUM_masked_band = band.copy()
        LUM_masked_band[soil_binary_mask] = np.nan
        LUM_masked_bands.append(LUM_masked_band)
    
    # Stack masked bands along the first axis to create a masked stacked image
    LUM_masked_map = np.stack(LUM_masked_bands, axis=0)
    LUM_masked_maps.append(LUM_masked_map)
    
    # Save the masked stacked image as a TIFF file
    location_name = f'location{location_index}'
    output_directory = os.path.join(output_parent_directory, location_name)
    os.makedirs(output_directory, exist_ok=True)
    
    # Save each band as a separate TIFF file
    for i, band_name in enumerate(band_names):
        output_image_path = os.path.join(output_directory, f'{location_name}_{band_name}_masked.tif')
        band_to_save = LUM_masked_map[i]
        # Example saving method using rasterio (adjust as needed)
        with rasterio.open(output_image_path, 'w', driver='GTiff', width=band_to_save.shape[1], height=band_to_save.shape[0], count=1, dtype=str(band_to_save.dtype), crs=clipped_meta['crs'], transform=clipped_transform) as dst:
            dst.write(band_to_save, 1)
    
    # Save the stacked image of all bands
    stacked_image_path = os.path.join(output_directory, f'{location_name}_stacked.tif')
    # Assuming you have a method to stack all bands (e.g., numpy stack)
    stacked_image = np.stack(bands, axis=0)
    # Example saving method for stacked image using rasterio (adjust as needed)
    with rasterio.open(stacked_image_path, 'w', driver='GTiff', width=stacked_image.shape[2], height=stacked_image.shape[1], count=stacked_image.shape[0], dtype=str(stacked_image.dtype), crs=clipped_meta['crs'], transform=clipped_transform) as dst:
        dst.write(stacked_image)

# Calculate the average, maximum, and minimum histograms
average_non_zero_histogram = accumulated_non_zero_histogram / num_locations
max_non_zero_histogram = np.max(individual_non_zero_histograms, axis=0)
min_non_zero_histogram = np.min(individual_non_zero_histograms, axis=0)

# Plot individual histograms and the average histogram with variance
plt.figure(figsize=(5, 3))
# Overlay Otsu threshold values as vertical lines
for threshold in LUM_otsu_thresholds:
    plt.axvline(x=threshold, color='gray', linestyle='-', linewidth=1, alpha=0.1)
    
# Plot the maximum and minimum histograms as a filled area
plt.fill_between(bin_edges_non_zero[:-1], min_non_zero_histogram, max_non_zero_histogram, color='green', alpha=0.5, label='Min-Max Range')

# Plot the average histogram
plt.plot(bin_edges_non_zero[:-1], average_non_zero_histogram, color='red', linewidth=2, label='Average Histogram')

# Overlay Otsu threshold values as vertical lines
for threshold in LUM_otsu_thresholds:
    plt.axvline(x=threshold, color='gray', linestyle='-', linewidth=0.1, alpha=0.1)

plt.xlim(0.1,0.7)
plt.xlabel('NIR band reflectance value')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

print("All masked stacked maps and stacked images saved.")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters


# Select the masked stacked image for the first location
masked_stacked_image_location = LUM_masked_maps[0]

# Define band names
band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']

# Create a figure with subplots for each band
num_bands = masked_stacked_image_location.shape[0]
fig, axes = plt.subplots(num_bands, 1, figsize=(8, 2*num_bands))

# Loop through each band and plot it
for i in range(num_bands):
    band = masked_stacked_image_location[i]
    ax = axes[i]
    im = ax.imshow(band, cmap='viridis', vmin=np.nanmin(band), vmax=np.nanmax(band))
    ax.set_title(f'{band_names[i]} Band with Soil Masking')
    ax.set_xlabel('Column Index')
    ax.set_ylabel('Row Index')
    fig.colorbar(im, ax=ax, label='Reflectance', orientation='vertical', pad=0.05)

plt.tight_layout()
plt.show()
